# Lab 1

In this assignment, you are going to use Apache Spark to explore the page views of Wikimedia projects. As a dataset, we use the page view statistics generated between 0-1am on Jan 1, 2016. Each line of the dataset, delimited by a white space and contains the statistics for one Wikimedia page. The schema looks as follows:
- Project code: The project identifier for each page.
- Page title: A string containing the title of the page.
- Page hits: Number of requests on the specific hour.
- Page size: Size of the page.

### Task 1 - Spark

First, create a new Spark Session and load the dataset as below:

In [1]:
import util.control.Breaks._
import org.apache.spark.sql.functions.col
import org.apache.spark.rdd.RDD

In [2]:
val pagecounts = sc.textFile("pagecounts.out")

pagecounts = pagecounts.out MapPartitionsRDD[1] at textFile at <console>:32


pagecounts.out MapPartitionsRDD[1] at textFile at <console>:32

1. Create a case class called Log using the four field names of the dataset.

In [3]:
case class Log(code: String, title: String, hits: Long, size: Long)   

defined class Log


2. Create a function that takes a string, split it by white space and converts it into a log object.

In [4]:
def convert_string(s: String): Log = { 

    val s_split = s.split(" +")
    if (s_split.count(_.length>=1) == 4){
        return Log(s_split(0).toString, s_split(1).toString, s_split(2).toLong, s_split(3).toLong);
        } 
    else {
        return Log(s_split(0).toString, "", s_split(1).toLong, s_split(2).toLong);
    }
}

convert_string: (s: String)Log


3. Create a function that takes an RDD[String] and returns an RDD[Log].

In [5]:
def convert_rdd(rdd: RDD[String]): RDD[Log] = {
    return rdd.map(convert_string)
    }

convert_rdd: (rdd: org.apache.spark.rdd.RDD[String])org.apache.spark.rdd.RDD[Log]


In [6]:
val rdd_pagecounts = convert_rdd(pagecounts)

rdd_pagecounts = MapPartitionsRDD[2] at map at <console>:35


MapPartitionsRDD[2] at map at <console>:35

In the remaining sections of this exercise, you have to make use of the RDD[Log] that you have created. For each of the questions below, implement a Scala function that takes as input an RDD[Log] and prints the requested values. You must include all of those results in your report.
1. Retrieve the first 15 records and print out the result.

In [7]:
for (x <- rdd_pagecounts.take(15)) {
   println(x)
}

Log(aa,271_a.C,1,4675)
Log(aa,Category:User_th,1,4770)
Log(aa,Chiron_Elias_Krase,1,4694)
Log(aa,Dassault_rafaele,2,9372)
Log(aa,E.Desv,1,4662)
Log(aa,File:Wiktionary-logo-en.png,1,10752)
Log(aa,Indonesian_Wikipedia,1,4679)
Log(aa,Main_Page,5,266946)
Log(aa,Requests_for_new_languages/Wikipedia_Banyumasan,1,4733)
Log(aa,Special:Contributions/203.144.160.245,1,5812)
Log(aa,Special:Contributions/5.232.61.79,1,5805)
Log(aa,Special:Contributions/Ayarportugal,1,5808)
Log(aa,Special:Contributions/Born2bgratis,1,5812)
Log(aa,Special:ListFiles/Betacommand,1,5035)
Log(aa,Special:ListFiles/Bohdan_p,1,5036)


2. Determine the number of records the dataset has in total.

In [8]:
val page_total = rdd_pagecounts.count()

page_total = 3324129


3324129

3. Compute the min, max, and average page size.

In [9]:
val page_size = rdd_pagecounts.map(x=>x.size)

val page_size_min = page_size.min()
val page_size_max = page_size.max()
val page_size_avg = page_size.mean()

page_size = MapPartitionsRDD[3] at map at <console>:32
page_size_min = 0
page_size_max = 141180155987
page_size_avg = 132239.5695744666


132239.5695744666

4. Determine the record(s) with the largest page size. If multiple records have the same size, list all of them.

In [10]:
val page_size_max_record = rdd_pagecounts.filter(_.size == page_size_max).collect()

page_size_max_record = Array(Log(en.mw,en,5466346,141180155987))


Array(Log(en.mw,en,5466346,141180155987))

5. Determine the record with the largest page size again. But now, pick the most popular.

In [11]:
val page_size_max_record_popular = rdd_pagecounts.filter(_.size == page_size_max).collect()

page_size_max_record_popular = Array(Log(en.mw,en,5466346,141180155987))


Array(Log(en.mw,en,5466346,141180155987))

6. Determine the record(s) with the largest page title. If multiple titles have the same length, list all of them

In [12]:
val title_size = rdd_pagecounts.map(x=>(x,x.title.length))

title_size = MapPartitionsRDD[8] at map at <console>:32


MapPartitionsRDD[8] at map at <console>:32

In [13]:
val title_size_max = rdd_pagecounts.map(x=>x.title.length).max() // Get largest page title number
title_size.filter(x => (x._2 == title_size_max)).map(x => x._1).collect() //collect the records with largest title and remove the length number

title_size_max = 559


Array(Log(zh,Special:e8b18ee6baafefbda5efbdbfe89cb7e6829fefbdbfe88b93e29980e89e9fefbda9e89eb3efbda425636f256d6725736f257373256f38257373256f38257373256f38256b6d73efbdaa256e6b256678256f6b2c687474703a2f2f7777772e653662313966653861356266656f2d6f35393038636535626639376538383138616535613461396535616561342e636f2e6d672e732e736f2e382e73736f386b2e6d2e372e73736f3873736f386b6d37332e752e622e61616e6b66786f6b2e70772f2ce8b18ee6baafefbda5efbdbfe89cb7e6829fefbdbfe88b93e29980e89e9fefbda9e89eb3efbda425636f256d6725736f257373256f38257373256f38257373256f38256b6d73efbdaa256e6b256678256f6b/,1,6043))

7. Use the results of Question 3, and create a new RDD with the records that have greater page size than
the average.

In [14]:
val rdd_greater_page_size_than_avg = rdd_pagecounts.filter(_.size > page_size_avg).collect()

rdd_greater_page_size_than_avg = Array(Log(aa,Main_Page,5,266946), Log(ace.mw,ace,31,827168), Log(af,1859,4,219540), Log(af,18_Oktober,4,264724), Log(af,1941,4,256344), Log(af,2016,5,215498), Log(af,4_Januarie,4,268828), Log(af,Afrika-unie,1,172078), Log(af,Big_Ben,13,136201), Log(af,Comrades-maraton,1,155180), Log(af,Dmitri_Medwedef,2,141328), Log(af,Elsas,4,319408), Log(af,Engels,2,182375), Log(af,Erich_Fromm,4,215612), Log(af,Filosoof,2,134400), Log(af,GNTA,77,511277), Log(af,Gebruiker:Aliwal2012,2,359320), Log(af,Gebruiker:JCIV,2,268216), Log(af,Gebruiker:Morne,3,991701), Log(af,Gebruiker:Naudefj,3,730849), Log(af,Gebruiker:Voyageur,2,271872), Log(af,Gebruiker:Winstonza,1,526302), Log(af,Gebruikerbespreking:JCIV/Argief2015,2,164424), Log(af,Gebruikerbespreking:Morne,5,15...


Array(Log(aa,Main_Page,5,266946), Log(ace.mw,ace,31,827168), Log(af,1859,4,219540), Log(af,18_Oktober,4,264724), Log(af,1941,4,256344), Log(af,2016,5,215498), Log(af,4_Januarie,4,268828), Log(af,Afrika-unie,1,172078), Log(af,Big_Ben,13,136201), Log(af,Comrades-maraton,1,155180), Log(af,Dmitri_Medwedef,2,141328), Log(af,Elsas,4,319408), Log(af,Engels,2,182375), Log(af,Erich_Fromm,4,215612), Log(af,Filosoof,2,134400), Log(af,GNTA,77,511277), Log(af,Gebruiker:Aliwal2012,2,359320), Log(af,Gebruiker:JCIV,2,268216), Log(af,Gebruiker:Morne,3,991701), Log(af,Gebruiker:Naudefj,3,730849), Log(af,Gebruiker:Voyageur,2,271872), Log(af,Gebruiker:Winstonza,1,526302), Log(af,Gebruikerbespreking:JCIV/Argief2015,2,164424), Log(af,Gebruikerbespreking:Morne,5,15...

8. Compute the total number of pageviews for each project (as the schema shows, the first field of each record contains the project code).

In [15]:
val project_hits_map = rdd_pagecounts.map(x => (x.code, x.hits))
val project_hits = project_hits_map.reduceByKey(_+_)

project_hits_map = MapPartitionsRDD[13] at map at <console>:32
project_hits = ShuffledRDD[14] at reduceByKey at <console>:33


ShuffledRDD[14] at reduceByKey at <console>:33

In [52]:
project_hits.collect()

Array((pt,113575), (it.s,1444), (sk.mw,9548), (ka.d,31), (ik.d,1), (frp.mw,11), (yi.mw,70), (az.q,9), (nv,25), (fr.v,264), (he.q,1), (ky.d,20), (eml.mw,30), (bxr,12), (jv,1245), (zh.voy,18), (ia.v,1), (kw,290), (br.mw,279), (beta.mw,104), (eo.d,367), (ak.d,4), (diq,268), (uk.voy,1), (stq.mw,39), (ast.d,161), (gl.q,30), (ss,1217), (mk.d,18), (tl,2782), (lt.b,46), (sk,5289), (test2,109), (kg,104), (tr.d,2477), (gu.d,2), (om.d,45), (oc,1447), (nl.voy,89), (eu.mw,716), (ii,8), (nv.mw,62), (kk.mw,707), (krc.mw,21), (os.mw,14), (iu.mw,81), (als,1054), (outreach.m,205), (pt.mw,196160), (lt.mw,6913), (id.s,152), (ln.d,9), (fr.mw,536978), (nn.v,2), (www.w,20989), (is.d,68), (kg.mw,9), (ko,1228), (ff.mw,8), (ss.mw,14), (tt.mw,233), (sg.mw,18), (pam,328), (tum,68), ...

9. Report the 10 most popular pageviews of all projects, sorted by the total number of hits.

In [21]:
project_hits.sortBy(_._2, false).take(10)

Array((en.mw,5466346), (en,4959090), (es.mw,695531), (ja.mw,611443), (de.mw,572119), (fr.mw,536978), (ru.mw,466742), (it.mw,400297), (de,315929), (commons.m,285796))

10. Determine the number of page titles that start with the article "The". How many of those page titles
are not part of the English project (Pages that are part of the English project have "en" as the first
field)?

In [22]:
val page_start_the = rdd_pagecounts.filter(_.title.startsWith("The")).count()

page_start_the = 45020


45020

In [23]:
val page_start_the_not_en = rdd_pagecounts.filter(_.title.startsWith("The")).filter(x=> !(x.code contains("en"))).count()

page_start_the_not_en = 9128


9128

11. Determine the percentage of pages that have only received a single page view in this one hour of log
data.

In [24]:
val pagecount_one_hit = rdd_pagecounts.filter(_.hits == 1).count()

pagecount_one_hit = 2558332


2558332

In [25]:
val percentage_one_hit = pagecount_one_hit.toFloat/page_total

percentage_one_hit = 0.76962477


0.76962477

12. Determine the number of unique terms appearing in the page titles. Note that in page titles, terms
are delimited by "_" instead of a white space. You can use any number of normalization steps (e.g.,
lowercasing, removal of non-alphanumeric characters).

In [26]:
val lower_title = rdd_pagecounts.map(x=>x.title.replaceAll("[^\\p{IsAlphabetic}\\p{IsDigit}\\{_}]", "").toLowerCase().split("_"))

lower_title = MapPartitionsRDD[29] at map at <console>:32


MapPartitionsRDD[29] at map at <console>:32

In [28]:
lower_title.take(10)

Array(Array(271, ac), Array(categoryuser, th), Array(chiron, elias, krase), Array(dassault, rafaele), Array(edesv), Array(filewiktionarylogoenpng), Array(indonesian, wikipedia), Array(main, page), Array(requests, for, new, languageswikipedia, banyumasan), Array(specialcontributions203144160245))

In [29]:
val distinct_term_number = lower_title.flatMap(x => x).distinct.count()

distinct_term_number = 1689818


1689818

13. Determine the most frequently occurring page title term in this dataset.

In [30]:
val lower_title_freq = lower_title.flatMap(x => x).map(x=>(x,1)).reduceByKey(_+_)

lower_title_freq = ShuffledRDD[36] at reduceByKey at <console>:32


ShuffledRDD[36] at reduceByKey at <console>:32

In [31]:
val most_freq_term = lower_title_freq.sortBy(_._2, false).map(_._1).take(1)

most_freq_term = Array(of)


Array(of)

### Task 2 - Spark SQL

First, convert the pagecounts from RDD[String] into DataFrame (hint: you may need to transform RDD[String] into RDD[Log] and then DataFrame).

In [32]:
val df = rdd_pagecounts.toDF("project", "title", "hits", "size")

df = [project: string, title: string ... 2 more fields]


[project: string, title: string ... 2 more fields]

In [33]:
df.printSchema()

root
 |-- project: string (nullable = true)
 |-- title: string (nullable = true)
 |-- hits: long (nullable = false)
 |-- size: long (nullable = false)



In [34]:
df.show(2)

+-------+----------------+----+----+
|project|           title|hits|size|
+-------+----------------+----+----+
|     aa|         271_a.C|   1|4675|
|     aa|Category:User_th|   1|4770|
+-------+----------------+----+----+
only showing top 2 rows



Compute the min, max, and average page size.

In [35]:
import org.apache.spark.sql.functions
import org.apache.spark.sql.functions.{min, max, avg}

In [36]:
df.select(max("size")).show(false)

+------------+
|max(size)   |
+------------+
|141180155987|
+------------+



In [37]:
df.select(min("size")).show(false)

+---------+
|min(size)|
+---------+
|0        |
+---------+



In [38]:
df.select(avg("size")).show(false)

+------------------+
|avg(size)         |
+------------------+
|132239.56957446598|
+------------------+



Determine the record with the largest page size again. But now, pick the most popular.

In [40]:
import org.apache.spark.sql.functions.desc
df.orderBy(desc("size")).take(1) // largest page size

Array([en.mw,en,5466346,141180155987])

In [9]:
df.orderBy($"size".desc, $"hits".desc).show(false) // Largest page size and most popular 

+-------+---------------------------------------------+-------+------------+
|project|title                                        |hits   |size        |
+-------+---------------------------------------------+-------+------------+
|en.mw  |en                                           |5466346|141180155987|
|ja.mw  |ja                                           |611443 |15021588551 |
|es.mw  |es                                           |695531 |12261337515 |
|ru.mw  |ru                                           |466742 |11847816616 |
|fr.mw  |fr                                           |536978 |11752030020 |
|de.mw  |de                                           |572119 |9523069696  |
|it.mw  |it                                           |400297 |8176042087  |
|en     |Main_Page                                    |257915 |4289970372  |
|pt.mw  |pt                                           |196160 |4029404403  |
|pl.mw  |pl                                           |176059 |2782453516  |

Use the results of Question 3, and create a new RDD with the records that have greater page size than the average.

In [41]:
import org.apache.spark.sql.types

In [42]:
val average = df.select(avg("size")).collect()(0)(0)

average = 132239.56957446598


132239.56957446598

In [43]:
import org.apache.spark.sql.functions._

val df_greater = df.filter(col("size") > lit(average)).orderBy($"size")
df_greater.show(false)

+-------+------------------------------------+----+------+
|project|title                               |hits|size  |
+-------+------------------------------------+----+------+
|de     |Tabakrauchen                        |2   |132241|
|pt     |Parse                               |1   |132241|
|en     |Yahoo_Serious                       |8   |132243|
|en     |City_of_Hope_National_Medical_Center|3   |132243|
|en     |Disgaea_5:_Alliance_of_Vengeance    |8   |132243|
|en     |File:Mamintb.PNG                    |4   |132244|
|en     |Procellariidae                      |3   |132244|
|en     |Nat_Geo_Wild                        |7   |132244|
|en     |Virtual_particle                    |6   |132245|
|en     |Cee-lo                              |11  |132245|
|en     |Jean_Marsh                          |7   |132246|
|en     |Varsity_Blues_(film)                |9   |132247|
|en     |Rare-earth_magnet                   |8   |132248|
|en     |Farmington_Hills,_Michigan          |4   |13224

df_greater = [project: string, title: string ... 2 more fields]


[project: string, title: string ... 2 more fields]

Determine the number of unique terms appearing in the page titles. Note that in page titles, terms are delimited by _ instead of a white space. You can use any number of normalization steps (e.g.,lowercasing, removal of non-alphanumeric characters).

In [23]:
import org.apache.spark.sql.functions.regexp_replace
import org.apache.spark.sql.functions.trim
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions.lower

In [44]:
import org.apache.spark.sql.functions.regexp_replace

In [45]:
// Only keep alphabetic, digit, and _
// Apply lowercasing
val df_cleaned = df.withColumn("title",regexp_replace($"title", "[^\\p{IsAlphabetic}\\p{IsDigit}\\{_}]", "")).withColumn("title", lower(col("title"))) 

df_cleaned = [project: string, title: string ... 2 more fields]


[project: string, title: string ... 2 more fields]

In [46]:
// Split on _
// Explode on array and get tokens
val df_split = df_cleaned.withColumn("title",split(col("title"),"_")).withColumn("token", explode(col("title"))) 

df_split = [project: string, title: array<string> ... 3 more fields]


[project: string, title: array<string> ... 3 more fields]

In [49]:
// Count unique words
val total_words = df_split.groupBy("token").count()

total_words = [token: string, count: bigint]


[token: string, count: bigint]

In [50]:
// Total amount of unique words
total_words.count()

1689818

Determine the most frequently occurring page title term in this dataset.

In [51]:
total_words.orderBy(desc("count")).take(1)(0)(0)

of